<a href="https://colab.research.google.com/github/saurabhtayal/Capstone-Project---Diabetic-Patients-Readmission-Prediction/blob/master/Baseline_Model_Comparisons_Saurabh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Model Raw

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, f1_score, confusion_matrix, classification_report

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/saurabhtayal/Capstone-Project---Diabetic-Patients-Readmission-Prediction/master/diabetic_data.csv')

In [3]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [4]:
X = pd.get_dummies(df.drop('readmitted', axis = 1), drop_first = True)
y = df['readmitted'].replace({'NO' : 0, '>30' : 0, '<30' : 1})

In [5]:
X.head()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,gender_Male,gender_Unknown/Invalid,age_[10-20),age_[20-30),age_[30-40),age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),weight_?,weight_[0-25),weight_[100-125),weight_[125-150),weight_[150-175),weight_[175-200),weight_[25-50),weight_[50-75),weight_[75-100),payer_code_BC,payer_code_CH,...,chlorpropamide_Steady,chlorpropamide_Up,glimepiride_No,glimepiride_Steady,glimepiride_Up,acetohexamide_Steady,glipizide_No,glipizide_Steady,glipizide_Up,glyburide_No,glyburide_Steady,glyburide_Up,tolbutamide_Steady,pioglitazone_No,pioglitazone_Steady,pioglitazone_Up,rosiglitazone_No,rosiglitazone_Steady,rosiglitazone_Up,acarbose_No,acarbose_Steady,acarbose_Up,miglitol_No,miglitol_Steady,miglitol_Up,troglitazone_Steady,tolazamide_Steady,tolazamide_Up,insulin_No,insulin_Steady,insulin_Up,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes
0,2278392,8222157,6,25,1,1,41,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
1,149190,55629189,1,1,7,3,59,0,18,0,0,0,9,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1
2,64410,86047875,1,1,7,2,11,5,13,2,0,1,6,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1
3,500364,82442376,1,1,7,2,44,1,16,0,0,0,7,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1
4,16680,42519267,1,1,7,1,51,0,8,0,0,0,5,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1


In [6]:
y.value_counts()

0    90409
1    11357
Name: readmitted, dtype: int64

In [7]:
X.shape

(101766, 2436)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.3)

In [9]:
lr = LogisticRegression(random_state = 0)

In [10]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.0

In [11]:
confusion_matrix(y_test, y_pred)

array([[27108,     0],
       [ 3422,     0]])

In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94     27108
           1       0.00      0.00      0.00      3422

    accuracy                           0.89     30530
   macro avg       0.44      0.50      0.47     30530
weighted avg       0.79      0.89      0.84     30530



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 2nd Iteration with some-preprocessed data

## Filling Not Available values in admission columns w.r.t emergencies relative to each other

In [13]:
df = pd.read_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/diabetes_half_preprocessed_2.csv')
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,preceding_year_visits
0,Caucasian,0,[0-10),Not Available,Not Available,Referral,1,41,0,1,0,0,0,Diabetes,Not Required,Not Required,1,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0
1,Caucasian,0,[10-20),Emergency,Discharged to home,Emergency,3,59,0,18,0,0,0,"Endocrine, Nutritional, Metabolic, Immunity",Diabetes,"Endocrine, Nutritional, Metabolic, Immunity",9,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0
2,AfricanAmerican,0,[20-30),Emergency,Discharged to home,Emergency,2,11,5,13,2,0,1,"Pregnancy, Childbirth",Diabetes,External causes of injury,6,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,1,0,3
3,Caucasian,1,[30-40),Emergency,Discharged to home,Emergency,2,44,1,16,0,0,0,Infectious and Parasitic,Diabetes,Circulatory,7,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0
4,Caucasian,1,[40-50),Emergency,Discharged to home,Emergency,1,51,0,8,0,0,0,Neoplasms,Neoplasms,Diabetes,5,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,1,1,0,0


In [14]:
df.shape

(99300, 44)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99300 entries, 0 to 99299
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      97070 non-null  object
 1   gender                    99300 non-null  int64 
 2   age                       99300 non-null  object
 3   admission_type_id         99300 non-null  object
 4   discharge_disposition_id  99300 non-null  object
 5   admission_source_id       99300 non-null  object
 6   time_in_hospital          99300 non-null  int64 
 7   num_lab_procedures        99300 non-null  int64 
 8   num_procedures            99300 non-null  int64 
 9   num_medications           99300 non-null  int64 
 10  number_outpatient         99300 non-null  int64 
 11  number_emergency          99300 non-null  int64 
 12  number_inpatient          99300 non-null  int64 
 13  diag_1                    99300 non-null  object
 14  diag_2                

In [16]:
for i in df.columns:
    print(i, '\n', df[i].unique(), '\n')

race 
 ['Caucasian' 'AfricanAmerican' nan 'Other' 'Asian' 'Hispanic'] 

gender 
 [0 1] 

age 
 ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)'] 

admission_type_id 
 ['Not Available' 'Emergency' 'Elective'] 

discharge_disposition_id 
 ['Not Available' 'Discharged to home'
 'Transferred to another medical facility'
 'Discharged to home with home health service' 'Left AMA'
 'Still patient/referred to this institution'] 

admission_source_id 
 ['Referral' 'Emergency' 'Transferred from another health care facility'
 'Not Available'] 

time_in_hospital 
 [ 1  3  2  4  5 13 12  9  7 10  6 11  8 14] 

num_lab_procedures 
 [ 41  59  11  44  51  31  70  73  68  33  47  62  60  55  49  75  45  29
  35  42  66  36  19  64  25  53  52  87  27  37  28  48  10   2  65  67
  40  54  58  57  43  32  83  34  39  69  38  72  22  96  46  78  56  61
  88  50   1  18  82   9  63  24  71  77  81  76  90  93   3 103  13  80
  85  16  15  12  30  23  17  

In [17]:
for i in ['admission_type_id', 'admission_source_id']:
    print(df[i].value_counts())

Emergency        70504
Elective         18659
Not Available    10137
Name: admission_type_id, dtype: int64
Emergency                                        55850
Referral                                         30418
Not Available                                     6853
Transferred from another health care facility     6179
Name: admission_source_id, dtype: int64


In [18]:
for i in range(len(df['admission_type_id'])):
    if df.loc[i, 'admission_type_id'] == 'Not Available' and df.loc[i, 'admission_source_id'] == 'Emergency':
        df.loc[i, 'admission_type_id'] = 'Emergency'
print(df['admission_type_id'].value_counts())

Emergency        72365
Elective         18659
Not Available     8276
Name: admission_type_id, dtype: int64


In [19]:
for i in range(len(df['admission_source_id'])):
    if df.loc[i, 'admission_source_id'] == 'Not Available' and df.loc[i, 'admission_type_id'] == 'Emergency':
        df.loc[i, 'admission_source_id'] = 'Emergency'
print(df['admission_source_id'].value_counts())

Emergency                                        56912
Referral                                         30418
Transferred from another health care facility     6179
Not Available                                     5791
Name: admission_source_id, dtype: int64


In [20]:
for i in range(len(df['admission_type_id'])):
    if df.loc[i, 'admission_type_id'] == 'Not Available' and df.loc[i, 'admission_source_id'] == 'Emergency':
        df.loc[i, 'admission_type_id'] = 'Emergency'
print(df['admission_type_id'].value_counts())

Emergency        72365
Elective         18659
Not Available     8276
Name: admission_type_id, dtype: int64


In [21]:
for i in range(len(df['admission_source_id'])):
    if df.loc[i, 'admission_source_id'] == 'Not Available' and df.loc[i, 'admission_type_id'] == 'Emergency':
        df.loc[i, 'admission_source_id'] = 'Emergency'
print(df['admission_source_id'].value_counts())

Emergency                                        56912
Referral                                         30418
Transferred from another health care facility     6179
Not Available                                     5791
Name: admission_source_id, dtype: int64


In [22]:
df[df['admission_type_id'] == 'Emergency']['admission_source_id'].value_counts()

Emergency                                        55991
Referral                                         11513
Transferred from another health care facility     4861
Name: admission_source_id, dtype: int64

In [23]:
df[df['admission_type_id'] == 'Elective']['admission_source_id'].value_counts()

Referral                                         16432
Transferred from another health care facility     1247
Emergency                                          921
Not Available                                       59
Name: admission_source_id, dtype: int64

In [24]:
df[df['admission_type_id'] == 'Not Available']['admission_source_id'].value_counts()

Not Available                                    5732
Referral                                         2473
Transferred from another health care facility      71
Name: admission_source_id, dtype: int64

In [25]:
df[df['admission_source_id'] == 'Emergency']['admission_type_id'].value_counts()

Emergency    55991
Elective       921
Name: admission_type_id, dtype: int64

In [26]:
df[df['admission_source_id'] == 'Referral']['admission_type_id'].value_counts()

Elective         16432
Emergency        11513
Not Available     2473
Name: admission_type_id, dtype: int64

In [27]:
df[df['admission_source_id'] == 'Transferred from another health care facility']['admission_type_id'].value_counts()

Emergency        4861
Elective         1247
Not Available      71
Name: admission_type_id, dtype: int64

In [28]:
df[df['admission_source_id'] == 'Not Available']['admission_type_id'].value_counts()

Not Available    5732
Elective           59
Name: admission_type_id, dtype: int64

## Checking Race Values

### Dropping Nan

In [29]:
df_2 = df.dropna()
df.shape, df_2.shape

((99300, 44), (97070, 44))

In [30]:
X = df_2.drop('readmitted', axis = 1)
X = pd.get_dummies(X, drop_first = True)

In [31]:
y = df_2['readmitted']

In [32]:
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### Relacing with Mode

In [33]:
df['race'].isna().sum()

2230

In [34]:
X = df.drop('readmitted', axis = 1)
X['race'] = X['race'].fillna(X['race'].mode()[0])
print(X['race'].isna().sum())
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']

0


In [35]:
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.013657957244655582

### Filling with probability ratio

In [36]:
df['race'].isna().sum()

2230

In [37]:
X = df.drop('readmitted', axis = 1)
vc = X['race'].value_counts(normalize = True)
miss = X['race'].isna()
X.loc[miss, 'race'] = np.random.choice(vc.index, size = miss.sum(), p = vc.values)
X['race'].isna().sum()

0

In [38]:
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01336104513064133

## Treating Race Column (Dropping Nans) before moving on Age Column

In [39]:
df['race'].isna().sum(), df.shape

(2230, (99300, 44))

In [40]:
df = df.dropna()
df['race'].isna().sum(), df.shape

(0, (97070, 44))

## Age Values

### Original Categories

In [41]:
X = df.drop('readmitted', 1)
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numerical Range of Original Categories

In [42]:
X = df.drop('readmitted', 1)
X['age'].value_counts()

[70-80)     24732
[60-70)     21562
[50-60)     16698
[80-90)     16042
[40-50)      9386
[30-40)      3688
[90-100)     2521
[20-30)      1601
[10-20)       682
[0-10)        158
Name: age, dtype: int64

In [43]:
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 10,
                             '[20-30)' : 20,
                             '[30-40)' : 30,
                             '[40-50)' : 40,
                             '[50-60)' : 50,
                             '[60-70)' : 60,
                             '[70-80)' : 70,
                             '[80-90)' : 80,
                             '[90-100)' : 90})

In [44]:
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numeircal Value of Original Categories

In [45]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 5,
                             '[10-20)' : 15,
                             '[20-30)' : 25,
                             '[30-40)' : 35,
                             '[40-50)' : 45,
                             '[50-60)' : 55,
                             '[60-70)' : 65,
                             '[70-80)' : 75,
                             '[80-90)' : 85,
                             '[90-100)' : 95})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.013550135501355014

Higher Numeical Values of Original Categories

In [46]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 20,
                             '[20-30)' : 30,
                             '[30-40)' : 40,
                             '[40-50)' : 50,
                             '[50-60)' : 60,
                             '[60-70)' : 70,
                             '[70-80)' : 80,
                             '[80-90)' : 90,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 5 Categories (0-20) (20-40) (40-60) (60-90) (90-100)

In [47]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : '[0-20)',
                             '[10-20)' : '[0-20)',
                             '[20-30)' : '[20-40)',
                             '[30-40)' : '[20-40)',
                             '[40-50)' : '[40-60)',
                             '[50-60)' : '[40-60)',
                             '[60-70)' : '[60-90)',
                             '[70-80)' : '[60-90)',
                             '[80-90)' : '[60-90)',
                             '[90-100)' : '[90-100)'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.013550135501355014

Lower Numerical Values

In [48]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 20,
                             '[30-40)' : 20,
                             '[40-50)' : 40,
                             '[50-60)' : 40,
                             '[60-70)' : 60,
                             '[70-80)' : 60,
                             '[80-90)' : 60,
                             '[90-100)' : 90})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numeircal Values

In [49]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 10,
                             '[20-30)' : 30,
                             '[30-40)' : 30,
                             '[40-50)' : 50,
                             '[50-60)' : 50,
                             '[60-70)' : 75,
                             '[70-80)' : 75,
                             '[80-90)' : 75,
                             '[90-100)' : 95})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.014453477868112014

Higher Numerical Values

In [50]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 20,
                             '[10-20)' : 20,
                             '[20-30)' : 40,
                             '[30-40)' : 40,
                             '[40-50)' : 60,
                             '[50-60)' : 60,
                             '[60-70)' : 90,
                             '[70-80)' : 90,
                             '[80-90)' : 90,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories (0-40) (40-90) (90-100)

In [51]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : '[0-40)',
                             '[10-20)' : '[0-40)',
                             '[20-30)' : '[0-40)',
                             '[30-40)' : '[0-40)',
                             '[40-50)' : '[40-90)',
                             '[50-60)' : '[40-90)',
                             '[60-70)' : '[40-90)',
                             '[70-80)' : '[40-90)',
                             '[80-90)' : '[40-90)',
                             '[90-100)' : '[90-100)'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numeical Values

In [52]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 0,
                             '[30-40)' : 0,
                             '[40-50)' : 40,
                             '[50-60)' : 40,
                             '[60-70)' : 40,
                             '[70-80)' : 40,
                             '[80-90)' : 40,
                             '[90-100)' : 90})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [53]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 20,
                             '[10-20)' : 20,
                             '[20-30)' : 20,
                             '[30-40)' : 20,
                             '[40-50)' : 65,
                             '[50-60)' : 65,
                             '[60-70)' : 65,
                             '[70-80)' : 65,
                             '[80-90)' : 65,
                             '[90-100)' : 95})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [54]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 40,
                             '[10-20)' : 40,
                             '[20-30)' : 40,
                             '[30-40)' : 40,
                             '[40-50)' : 90,
                             '[50-60)' : 90,
                             '[60-70)' : 90,
                             '[70-80)' : 90,
                             '[80-90)' : 90,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories (0-50) (50-90) (90-100)

In [55]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : '[0-50)',
                             '[10-20)' : '[0-50)',
                             '[20-30)' : '[0-50)',
                             '[30-40)' : '[0-50)',
                             '[40-50)' : '[0-50)',
                             '[50-60)' : '[50-90)',
                             '[60-70)' : '[50-90)',
                             '[70-80)' : '[50-90)',
                             '[80-90)' : '[50-90)',
                             '[90-100)' : '[90-100)'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numeical Values

In [56]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 0,
                             '[30-40)' : 0,
                             '[40-50)' : 0,
                             '[50-60)' : 50,
                             '[60-70)' : 50,
                             '[70-80)' : 50,
                             '[80-90)' : 50,
                             '[90-100)' : 90})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [57]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 25,
                             '[10-20)' : 25,
                             '[20-30)' : 25,
                             '[30-40)' : 25,
                             '[40-50)' : 25,
                             '[50-60)' : 70,
                             '[60-70)' : 70,
                             '[70-80)' : 70,
                             '[80-90)' : 70,
                             '[90-100)' : 95})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [58]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 50,
                             '[10-20)' : 50,
                             '[20-30)' : 50,
                             '[30-40)' : 50,
                             '[40-50)' : 50,
                             '[50-60)' : 90,
                             '[60-70)' : 90,
                             '[70-80)' : 90,
                             '[80-90)' : 90,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 4 Categories According to Age Range - Child (0-10), Teen (10-20), Adult (20-60), Old (60-100)

In [59]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'Child',
                             '[10-20)' : 'Teen',
                             '[20-30)' : 'Adult',
                             '[30-40)' : 'Adult',
                             '[40-50)' : 'Adult',
                             '[50-60)' : 'Adult',
                             '[60-70)' : 'Old',
                             '[70-80)' : 'Old',
                             '[80-90)' : 'Old',
                             '[90-100)' : 'Old'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numeical Values

In [60]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 10,
                             '[20-30)' : 20,
                             '[30-40)' : 20,
                             '[40-50)' : 20,
                             '[50-60)' : 20,
                             '[60-70)' : 60,
                             '[70-80)' : 60,
                             '[80-90)' : 60,
                             '[90-100)' : 60})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [61]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 5,
                             '[10-20)' : 15,
                             '[20-30)' : 40,
                             '[30-40)' : 40,
                             '[40-50)' : 40,
                             '[50-60)' : 40,
                             '[60-70)' : 80,
                             '[70-80)' : 80,
                             '[80-90)' : 80,
                             '[90-100)' : 80})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [62]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 20,
                             '[20-30)' : 60,
                             '[30-40)' : 60,
                             '[40-50)' : 60,
                             '[50-60)' : 60,
                             '[60-70)' : 100,
                             '[70-80)' : 100,
                             '[80-90)' : 100,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories According to Age Range - Young (0-20), Adult (20-60), Old (60-100)

In [63]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'Young',
                             '[10-20)' : 'Young',
                             '[20-30)' : 'Adult',
                             '[30-40)' : 'Adult',
                             '[40-50)' : 'Adult',
                             '[50-60)' : 'Adult',
                             '[60-70)' : 'Old',
                             '[70-80)' : 'Old',
                             '[80-90)' : 'Old',
                             '[90-100)' : 'Old'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numeical Values

In [64]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 20,
                             '[30-40)' : 20,
                             '[40-50)' : 20,
                             '[50-60)' : 20,
                             '[60-70)' : 60,
                             '[70-80)' : 60,
                             '[80-90)' : 60,
                             '[90-100)' : 60})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [65]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 10,
                             '[20-30)' : 40,
                             '[30-40)' : 40,
                             '[40-50)' : 40,
                             '[50-60)' : 40,
                             '[60-70)' : 80,
                             '[70-80)' : 80,
                             '[80-90)' : 80,
                             '[90-100)' : 80})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [66]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 20,
                             '[10-20)' : 20,
                             '[20-30)' : 60,
                             '[30-40)' : 60,
                             '[40-50)' : 60,
                             '[50-60)' : 60,
                             '[60-70)' : 100,
                             '[70-80)' : 100,
                             '[80-90)' : 100,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories According to Age Range - Young (0-30), Mid (30-60), Old (60-100)

In [67]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'Young',
                             '[10-20)' : 'Young',
                             '[20-30)' : 'Young',
                             '[30-40)' : 'Mid',
                             '[40-50)' : 'Mid',
                             '[50-60)' : 'Mid',
                             '[60-70)' : 'Old',
                             '[70-80)' : 'Old',
                             '[80-90)' : 'Old',
                             '[90-100)' : 'Old'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.014453477868112014

Lower Numeical Values

In [68]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 0,
                             '[30-40)' : 30,
                             '[40-50)' : 30,
                             '[50-60)' : 30,
                             '[60-70)' : 60,
                             '[70-80)' : 60,
                             '[80-90)' : 60,
                             '[90-100)' : 60})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [69]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 15,
                             '[10-20)' : 15,
                             '[20-30)' : 15,
                             '[30-40)' : 45,
                             '[40-50)' : 45,
                             '[50-60)' : 45,
                             '[60-70)' : 80,
                             '[70-80)' : 80,
                             '[80-90)' : 80,
                             '[90-100)' : 80})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [70]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 30,
                             '[10-20)' : 30,
                             '[20-30)' : 30,
                             '[30-40)' : 60,
                             '[40-50)' : 60,
                             '[50-60)' : 60,
                             '[60-70)' : 100,
                             '[70-80)' : 100,
                             '[80-90)' : 100,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 5 Categories According to Readmission Rate

In [71]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'A',
                             '[10-20)' : 'A',
                             '[20-30)' : 'B',
                             '[30-40)' : 'B',
                             '[40-50)' : 'C',
                             '[50-60)' : 'C',
                             '[60-70)' : 'D',
                             '[70-80)' : 'E',
                             '[80-90)' : 'D',
                             '[90-100)' : 'B'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 4 Categories According to Readmission Rate

In [72]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'A',
                             '[10-20)' : 'A',
                             '[20-30)' : 'B',
                             '[30-40)' : 'B',
                             '[40-50)' : 'C',
                             '[50-60)' : 'C',
                             '[60-70)' : 'D',
                             '[70-80)' : 'D',
                             '[80-90)' : 'D',
                             '[90-100)' : 'B'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories According to Readmission Rate

In [73]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'A',
                             '[10-20)' : 'A',
                             '[20-30)' : 'A',
                             '[30-40)' : 'A',
                             '[40-50)' : 'B',
                             '[50-60)' : 'B',
                             '[60-70)' : 'C',
                             '[70-80)' : 'C',
                             '[80-90)' : 'C',
                             '[90-100)' : 'A'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

## Treating Age column (Taking Middle Numerical value after making 5 categories)

In [74]:
df['age'] = df['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 10,
                             '[20-30)' : 30,
                             '[30-40)' : 30,
                             '[40-50)' : 50,
                             '[50-60)' : 50,
                             '[60-70)' : 75,
                             '[70-80)' : 75,
                             '[80-90)' : 75,
                             '[90-100)' : 95})
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,preceding_year_visits
0,Caucasian,0,10,Not Available,Not Available,Referral,1,41,0,1,0,0,0,Diabetes,Not Required,Not Required,1,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0
1,Caucasian,0,10,Emergency,Discharged to home,Emergency,3,59,0,18,0,0,0,"Endocrine, Nutritional, Metabolic, Immunity",Diabetes,"Endocrine, Nutritional, Metabolic, Immunity",9,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0
2,AfricanAmerican,0,30,Emergency,Discharged to home,Emergency,2,11,5,13,2,0,1,"Pregnancy, Childbirth",Diabetes,External causes of injury,6,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,1,0,3
3,Caucasian,1,30,Emergency,Discharged to home,Emergency,2,44,1,16,0,0,0,Infectious and Parasitic,Diabetes,Circulatory,7,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0
4,Caucasian,1,50,Emergency,Discharged to home,Emergency,1,51,0,8,0,0,0,Neoplasms,Neoplasms,Diabetes,5,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,1,1,0,0


In [75]:
df.shape

(97070, 44)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97070 entries, 0 to 99299
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      97070 non-null  object
 1   gender                    97070 non-null  int64 
 2   age                       97070 non-null  int64 
 3   admission_type_id         97070 non-null  object
 4   discharge_disposition_id  97070 non-null  object
 5   admission_source_id       97070 non-null  object
 6   time_in_hospital          97070 non-null  int64 
 7   num_lab_procedures        97070 non-null  int64 
 8   num_procedures            97070 non-null  int64 
 9   num_medications           97070 non-null  int64 
 10  number_outpatient         97070 non-null  int64 
 11  number_emergency          97070 non-null  int64 
 12  number_inpatient          97070 non-null  int64 
 13  diag_1                    97070 non-null  object
 14  diag_2                

In [77]:
df.describe(include = 'all')

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,preceding_year_visits
count,97070,97070.000000,97070.000000,97070,97070,97070,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070,97070,97070,97070.000000,97070,97070,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000,97070.0,97070.000000,97070.000000,97070.000000,97070.000000,97070.000000
unique,5,NaN,NaN,3,6,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,20,20,NaN,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Caucasian,NaN,NaN,Emergency,Discharged to home,Emergency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Circulatory,Circulatory,Circulatory,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,74201,NaN,NaN,70595,58751,55994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29004,30463,28947,NaN,91978,80662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.460997,65.787215,NaN,NaN,NaN,4.381488,42.882775,1.335325,15.984928,0.373359,0.202009,0.638488,NaN,NaN,NaN,7.418955,NaN,NaN,-1.595962,-1.968281,-1.985845,-1.998238,-1.896333,-1.999979,-1.745524,-1.786329,-1.999588,-1.852581,-1.872134,-1.993767,-1.999248,-1.999938,-1.999186,-0.947553,-1.986144,-1.999732,-1.999979,-2.0,-1.999979,0.463830,0.771011,0.114557,1.213856
std,NaN,0.498479,15.503470,NaN,NaN,NaN,2.970267,19.629269,1.700854,8.087074,1.274882,0.947541,1.269112,NaN,NaN,NaN,1.933679,NaN,NaN,0.819173,0.255473,0.169431,0.060789,0.452185,0.006419,0.679585,0.632697,0.028705,0.528109,0.493693,0.112715,0.038908,0.011118,0.040718,1.099084,0.166357,0.023144,0.006419,0.0,0.006419,0.498693,0.420185,0.318488,2.311595
min,NaN,0.000000,10.000000,NaN,NaN,NaN,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,1.000000,NaN,NaN,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.0,-2.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,0.000000,50.000000,NaN,NaN,NaN,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,6.000000,NaN,NaN,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.0,-2.000000,0.000000,1.000000,0.000000,0.000000
50%,NaN,0.000000,75.000000,NaN,NaN,NaN,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,8.000000,NaN,NaN,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-1.000000,-2.000000,-2.000000,-2.000000,-2.0,-2.000000,0.000000,1.000000,0.000000,0.000000
75%,NaN,1.000000,75.000000,NaN,NaN,NaN,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,NaN,NaN,NaN,9.000000,NaN,NaN,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,0.000000,-2.000000,-2.000000,-2.000000,-2.0,-2.000000,1.000000,1.000000,0.000000,2.000000


In [78]:
for i in df.columns:
    print(i, '\n', df[i].value_counts(), '\n')

race 
 Caucasian          74201
AfricanAmerican    18759
Hispanic            2012
Other               1471
Asian                627
Name: race, dtype: int64 

gender 
 0    52321
1    44749
Name: gender, dtype: int64 

age 
 75    62336
50    26084
30     5289
95     2521
10      840
Name: age, dtype: int64 

admission_type_id 
 Emergency        70595
Elective         18305
Not Available     8170
Name: admission_type_id, dtype: int64 

discharge_disposition_id 
 Discharged to home                             58751
Transferred to another medical facility        20167
Discharged to home with home health service    12797
Not Available                                   4644
Left AMA                                         610
Still patient/referred to this institution       101
Name: discharge_disposition_id, dtype: int64 

admission_source_id 
 Emergency                                        55994
Referral                                         29653
Not Available                       

In [81]:
df.to_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/Diabetes_Preprocessed.csv', index = False)